In [3]:
# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import pyplot
%matplotlib inline

In [4]:
#Hallo Herr Zimmermann,

 

#vielen Dank für Ihre Nachricht.

#Ich habe mir das Notebook angeschaut und eine Lösung vorgeschlagen. Allerdings verstehe ich das Ziel der Methode nicht ganz, von daher müssten Sie einmal prüfen, ob ich es richtig interpretiert habe.

#Mein Vorschlag kann folgendes: für ein Fenster von z.B. 10 Nachrichten vorher oder nachher (Achtung: nachher ist für eine Prediction nicht verwendbar! Zukunftsinformation nennt man das) wird gezählt, wie oft die severity-Klassen 1, 2 und 3 vorkommen. Dies lege ich in drei Spalten ab: pre_no_srv_1, pre_no_srv_2, pre_no_srv_3

#Die vielen Spalten, die durch das „Shiften“ zustande kommen, können Sie auch wieder löschen. Diese sind nur für den Zwischenschritt nötig.

#So bewege ich mich dauerhaft in der Pandas-Welt und von der Performance (Laufzeit und Speicher) gibt es keine Problem.

 

#Ist es das, was Sie erreichen wollten?

 

#Hinsichtlich der Mappings habe ich Ihnen auch noch einen Vorschlag gemacht.

 

#Viele Grüße und ein schönes Wochenende

#Peer Küppers 

In [5]:
#Daten einlesen
df_orig = pd.read_parquet("D:/Eclipse Python/DataScience/res/event_ano.parquet.gzip")

In [6]:
df = df_orig.set_index("event_timestamp")
del df['Unnamed: 0']
df.sort_index(inplace=True)

In [7]:
line_mapping = {}
for k, v in [(l, int(l.split("_")[1])) for l in list(df.an_line.unique())]:
    line_mapping[k] = v

df["an_line_no"] = df.an_line.map(line_mapping)  

In [8]:
cell_mapping = {}
for k, v in [(l, int(l.split("_")[1])) for l in list(df.an_cell.unique())]:
    cell_mapping[k] = v

df["an_cell_no"] = df.an_cell.map(cell_mapping)  

In [9]:
robot_mapping = {}
for k, v in [(l, int(l.split("_")[1])) for l in list(df.an_robot.unique())]:
    robot_mapping[k] = v

df["an_robot_no"] = df.an_robot.map(robot_mapping)  

In [10]:
message_severity_mapping = {"Error":1,"Warning":2,"Information":3}

df["message_severity"] = df.message_severity.apply(lambda x: message_severity_mapping[x] if x in message_severity_mapping.keys() else x)

In [11]:
def dfExtractMethod(df,extract_value):
    df_extract = df.loc[df['an_line_no'] == extract_value]
    return df_extract

In [50]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternAllMethod(df,no_of_messages):
    dataFrame = pd.DataFrame()
    
    for j in range(0,28):
        df_extract = dfExtractMethod(df,j)
        df_extract = df_extract.reset_index()
                
        for i in range(1,no_of_messages + 1):
            df_extract['msg_svr_pre_{}'.format(i)] = df_extract.shift(i).message_severity

        pre_column_names = ['msg_svr_pre_{}'.format(i) for i in range(1, no_of_messages + 1)]

        for i in range(1, 4):
            df_extract['pre_no_srv_{}'.format(i)] = (df_extract[pre_column_names]==i).sum(axis=1)
        
        dataFrame = dataFrame.append(df_extract)
    return dataFrame

In [51]:
df_ErrorsPerLinesPattern = dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternAllMethod(df,10)
df_ErrorsPerLinesPattern.to_parquet("D:/Eclipse Python/DataScience/res/event_ano_pattern_all.parquet.gzip", compression='gzip')

In [27]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternSinglesMethod(df,no_of_messages):
    for j in range(0,28):
        df_extract = dfExtractMethod(df,j)
        df_extract = df_extract.reset_index()
        
        for i in range(1,no_of_messages + 1):
            df_extract['msg_svr_pre_{}'.format(i)] = df_extract.shift(i).message_severity

        pre_column_names = ['msg_svr_pre_{}'.format(i) for i in range(1, no_of_messages + 1)]

        for i in range(1, 4):
            df_extract['pre_no_srv_{}'.format(i)] = (df_extract[pre_column_names]==i).sum(axis=1)
    
        df_extract.to_parquet("D:/Eclipse Python/DataScience/res/event_ano_pattern_single_{}.parquet.gzip".format(j),compression='gzip')

In [28]:
dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternSinglesMethod(df,10)

In [54]:
def dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternDataSetMethod(df,no_of_messages):
    for j in range(0,28):
        df_extract = dfExtractMethod(df,j)
        df_extract = df_extract.reset_index()
        
        for i in range(1,no_of_messages + 1):
            if(df_extract.shift(i).message_severity == 1.0):
                print(df_extract.shift(i))

        
        #df_extract.to_parquet("D:/Eclipse Python/DataScience/res/event_ano_pattern_DataSet.parquet.gzip".format(j), compression='gzip')

In [55]:
dfExtractErrorWarningInformationAnd10RowsUpAndDownPatternDataSetMethod(df,10)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().